# 資料寫入 BigQuery

https://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_hedge_result.php?l=zh-tw&o=htm&se=AL&t=D&d=113/01/05&s=0,asc

## 本益比資料

In [77]:
import requests
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
from datetime import datetime, timedelta

# 指定時間區段
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 1, 8)

# 創建一個空的 DataFrame 用來存放資料
df = pd.DataFrame()

# 生成日期範圍
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

# 逐一抓取每天的資料
for date in date_range:
    formatted_date = date.strftime('%Y%m%d')
    url = f'https://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_hedge_result.php?l=zh-tw&o=htm&se=AL&t=D&d={str(int(formatted_date[:4])-1911)}/{formatted_date[4:6]}/{formatted_date[6:8]}&s=0,asc'
    response = requests.get(url)
    if response.status_code == 200:
        # 使用BeautifulSoup解析HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # 在這裡找到表格部分
        table = soup.find('table')
        # 確保找到了表格
        if table:
            # 提取表格數據
            df_daily = pd.read_html(str(table), header=1)[0]
            df_daily['Date'] = date  # 加入日期欄位
            df = pd.concat([df, df_daily], ignore_index=True)
            print(f'Processing data for {formatted_date}')

# 顯示合併後的 DataFrame
df = df[['Date', '代號', '名稱', '外資及陸資(不含外資自營商)', '外資及陸資(不含外資自營商).1', '外資及陸資(不含外資自營商).2',
       '外資自營商', '外資自營商.1', '外資自營商.2', '外資及陸資', '外資及陸資.1', '外資及陸資.2', '投信',
       '投信.1', '投信.2', '自營商(自行買賣)', '自營商(自行買賣).1', '自營商(自行買賣).2', '自營商(避險)',
       '自營商(避險).1', '自營商(避險).2', '自營商', '自營商.1', '自營商.2', '三大法人買賣超 股數合計']].iloc[1:]
# df.columns = ['date', 'stock_id', '名稱', '外資及陸資(不含外資自營商)買進股數', '外資及陸資(不含外資自營商)賣出股數', '外資及陸資(不含外資自營商)買賣超股數',
#        '外資自營商買進股數', '外資自營商賣出股數', '外資自營商買賣超股數', '外資及陸資買進股數', '外資及陸資賣出股數', '外資及陸資買賣超股數', '投信買進股數',
#        '投信賣出股數', '投信買賣超股數', '自營商(自行買賣)買進股數', '自營商(自行買賣)賣出股數', '自營商(自行買賣)買賣超股數', '自營商(避險)買進股數',
#        '自營商(避險)賣出股數', '自營商(避險)買賣超股數', '自營商買進股數', '自營商賣出股數', '自營商買賣超股數', '三大法人買賣超 股數合計']

df.columns = ['date', 'stock_id', 'name', 'foreign_and_mainland_buy_volume', 'foreign_and_mainland_sell_volume', 'foreign_and_mainland_net_volume',
              'foreign_self_buy_volume', 'foreign_self_sell_volume', 'foreign_self_net_volume', 'foreign_and_mainland_total_buy_volume', 'foreign_and_mainland_total_sell_volume', 'foreign_and_mainland_total_net_volume',
              'investment_trust_buy_volume', 'investment_trust_sell_volume', 'investment_trust_net_volume', 'proprietary_trader_self_buy_volume', 'proprietary_trader_self_sell_volume', 'proprietary_trader_self_net_volume',
              'proprietary_trader_hedge_buy_volume', 'proprietary_trader_hedge_sell_volume', 'proprietary_trader_hedge_net_volume', 'proprietary_trader_total_buy_volume', 'proprietary_trader_total_sell_volume', 'proprietary_trader_total_net_volume',
              'three_major_institution_net_volume']

Processing data for 20240101
Processing data for 20240102
Processing data for 20240103
Processing data for 20240104
Processing data for 20240105
Processing data for 20240106
Processing data for 20240107
Processing data for 20240108


In [78]:
df['date'].unique()

array(['2024-01-01T00:00:00.000000000', '2024-01-02T00:00:00.000000000',
       '2024-01-03T00:00:00.000000000', '2024-01-04T00:00:00.000000000',
       '2024-01-05T00:00:00.000000000', '2024-01-06T00:00:00.000000000',
       '2024-01-07T00:00:00.000000000', '2024-01-08T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [79]:
df.shape

(23973, 25)

# 篩選特定公司

In [80]:
import pandas as pd
from google.cloud import bigquery
from pandas_gbq import to_gbq, read_gbq
from google.colab import drive
from google.oauth2 import service_account

drive.mount('/content/gdrive')

# 設定你的Google Cloud帳戶認證檔案路徑
credentials_path = '/content/gdrive/My Drive/tw-stock.json'

# 設定你的BigQuery專案ID
project_id = 'tw-stock-410406'

# Load credentials using google.oauth2.service_account
credentials_obj = service_account.Credentials.from_service_account_file(credentials_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [81]:
company_info_df = read_gbq(f'SELECT * FROM {project_id}.financial_data.company_info', project_id=project_id, credentials=credentials_obj, location='US')
# company_info_df.head()


Downloading:   0%|          |
Downloading: 100%|██████████|


In [82]:
company_list = company_info_df['stock_id'].unique()
filtered_data = df[df['stock_id'].isin(company_list)]

In [83]:
filtered_data.tail(3)

,date,stock_id,name,foreign_and_mainland_buy_volume,foreign_and_mainland_sell_volume,foreign_and_mainland_net_volume,foreign_self_buy_volume,foreign_self_sell_volume,foreign_self_net_volume,foreign_and_mainland_total_buy_volume,...,proprietary_trader_self_buy_volume,proprietary_trader_self_sell_volume,proprietary_trader_self_net_volume,proprietary_trader_hedge_buy_volume,proprietary_trader_hedge_sell_volume,proprietary_trader_hedge_net_volume,proprietary_trader_total_buy_volume,proprietary_trader_total_sell_volume,proprietary_trader_total_net_volume,three_major_institution_net_volume
19817,2024-01-08,9950,萬國通,77000,15000,62000,0,0,0,77000,...,0,0,0,0,0,0,0,0,0,62000
19818,2024-01-08,9951,皇田,44000,79000,-35000,0,0,0,44000,...,0,0,0,0,0,0,0,0,0,-35000
19819,2024-01-08,9962,有益,16000,263000,-247000,0,0,0,16000,...,0,0,0,0,0,0,0,0,0,-247000


In [84]:
filtered_data.shape

(3214, 25)

# 寫入資料庫

In [ ]:
# 從BigQuery中讀取資料
read_df = read_gbq(f'SELECT COUNT(*) as row_count FROM {project_id}.financial_data.institutional_investors', project_id=project_id, credentials=credentials_obj, location='US')
read_df['row_count'][0]

Downloading: 100%|██████████|


3072021

In [ ]:
import pandas as pd
from google.cloud import bigquery
from pandas_gbq import to_gbq, read_gbq
from google.colab import drive
from google.oauth2 import service_account

drive.mount('/content/gdrive')

# 設定你的Google Cloud帳戶認證檔案路徑
credentials_path = '/content/gdrive/My Drive/tw-stock.json'

# 設定你的BigQuery專案ID
project_id = 'tw-stock-410406'

# Load credentials using google.oauth2.service_account
credentials_obj = service_account.Credentials.from_service_account_file(credentials_path)

# 讀取目前表格的資料
current_data = read_gbq(f'SELECT COUNT(*) as row_count FROM {project_id}.financial_data.institutional_investors_2024_OCT', project_id=project_id, credentials=credentials_obj, location='US')
current_data_count = current_data['row_count'][0]

# 將DataFrame寫入BigQuery
to_gbq(df, destination_table=f'{project_id}.financial_data.institutional_investors_2024_OCT', project_id=project_id, if_exists='append', credentials=credentials_obj, location='US')

# 驗證資料筆數
new_data = read_gbq(f'SELECT COUNT(*) as row_count FROM {project_id}.financial_data.institutional_investors_2024_OCT', project_id=project_id, credentials=credentials_obj, location='US')
new_data_count = new_data['row_count'][0]

# 判斷是否寫入
if current_data_count + len(df) == new_data_count:
    print("資料寫入成功")
    print('寫入前資料數：', current_data_count)
    print('寫後資料數：', len(df))
    print('寫入後資料數：', new_data_count)
else:
    print("資料寫入失敗，筆數不符合預期")
    print('寫入前資料數：', current_data_count)
    print('寫後資料數：', len(df))
    print('寫入後資料數：', new_data_count)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Downloading: 100%|██████████|


100%|██████████| 1/1 [00:00<00:00, 5769.33it/s]


Downloading: 100%|██████████|
資料寫入成功
寫入前資料數： 8334
寫後資料數： 2778
寫入後資料數： 11112


# 初次寫入

In [85]:
# import pandas as pd
# from google.cloud import bigquery
# from pandas_gbq import to_gbq, read_gbq
# from google.colab import drive
# from google.oauth2 import service_account

# drive.mount('/content/gdrive')

# 設定你的Google Cloud帳戶認證檔案路徑
credentials_path = '/content/gdrive/My Drive/tw-stock.json'

# 設定你的BigQuery專案ID
project_id = 'tw-stock-410406'

# Load credentials using google.oauth2.service_account
credentials_obj = service_account.Credentials.from_service_account_file(credentials_path)
# 將DataFrame寫入BigQuery
to_gbq(filtered_data, destination_table=f'{project_id}.financial_data.institutional_investors_2024_OCT', project_id=project_id, if_exists='replace', credentials=credentials_obj, location='US')


100%|██████████| 1/1 [00:00<00:00, 6442.86it/s]


In [86]:
# 從BigQuery中讀取資料
read_df = read_gbq(f'SELECT COUNT(*) as row_count FROM {project_id}.financial_data.institutional_investors_2024_OCT', project_id=project_id, credentials=credentials_obj, location='US')
read_df['row_count'][0]


Downloading:   0%|          |
Downloading: 100%|██████████|


3214

In [87]:
# 從BigQuery中讀取資料
read_df = read_gbq(f'SELECT * FROM {project_id}.financial_data.institutional_investors_2024_OCT', project_id=project_id, credentials=credentials_obj, location='US')
read_df.tail(3)


Downloading:   0%|          |
Downloading: 100%|██████████|


,date,stock_id,name,foreign_and_mainland_buy_volume,foreign_and_mainland_sell_volume,foreign_and_mainland_net_volume,foreign_self_buy_volume,foreign_self_sell_volume,foreign_self_net_volume,foreign_and_mainland_total_buy_volume,...,proprietary_trader_self_buy_volume,proprietary_trader_self_sell_volume,proprietary_trader_self_net_volume,proprietary_trader_hedge_buy_volume,proprietary_trader_hedge_sell_volume,proprietary_trader_hedge_net_volume,proprietary_trader_total_buy_volume,proprietary_trader_total_sell_volume,proprietary_trader_total_net_volume,three_major_institution_net_volume
3211,2024-01-05 00:00:00+00:00,8354,冠好,0,6000,-6000,0,0,0,0,...,0,1000,-1000,0,0,0,0,1000,-1000,-7000
3212,2024-01-05 00:00:00+00:00,8390,金益鼎,22000,86000,-64000,0,0,0,22000,...,0,0,0,0,1000,-1000,0,1000,-1000,-65000
3213,2024-01-05 00:00:00+00:00,8927,北基,47000,63000,-16000,0,0,0,47000,...,0,0,0,0,1000,-1000,0,1000,-1000,-17000
